In [1]:
import xgboost as xgb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
import datetime
import sys
from tqdm import tqdm
import os
from tqdm import tqdm
import random
import pickle
from glob import glob
import gc
from multiprocessing import Pool, cpu_count

import warnings
warnings.filterwarnings('ignore')

sys.path.append('/kaggle/src')
from utils.xgb import fit_xgb, inference_xgb
from utils.metric import compute_comptetition_metric
from utils.feature_contena import Features
from utils.pandas_utils import reduce_mem_usage

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
config = f"""
execution:
    exp_id: exp_093
    debug: False
    submit: False
    multiprocessing: True

dataset: 
    competition_dir: /kaggle/input/child-mind-institute-detect-sleep-states
    cv_split_path: /kaggle/input/cv_split/train_folds.csv
    train_base_path: /kaggle/input/train_base/train_base.csv
    step_csv_dir: /kaggle/input/save_series_csv/csvs

feature:
    agg_freq: 24 # [step]

seed: 46
"""

CFG = yaml.load(config, Loader=yaml.SafeLoader)

CFG["output_dir"] = f"/kaggle/output/{CFG['execution']['exp_id']}"
os.makedirs(CFG["output_dir"], exist_ok=True)

In [3]:
oof = pd.read_parquet(os.path.join("/kaggle/output", os.path.join(CFG["execution"]["exp_id"], "oof.parquet")))
sub = pd.read_csv(os.path.join("/kaggle/output", os.path.join(CFG["execution"]["exp_id"], "submission.csv")))
display(oof.head())

,series_id,step,wakeup_oof,onset_oof
0,05e1944c3818,6,0.004084,0.002020
1,05e1944c3818,18,0.003406,0.001832
2,05e1944c3818,30,0.003180,0.001751
3,05e1944c3818,42,0.003114,0.001701
4,05e1944c3818,54,0.003104,0.001665


In [4]:
oof

,series_id,step,wakeup_oof,onset_oof
0,05e1944c3818,6,0.004084,0.002020
1,05e1944c3818,18,0.003406,0.001832
2,05e1944c3818,30,0.003180,0.001751
3,05e1944c3818,42,0.003114,0.001701
4,05e1944c3818,54,0.003104,0.001665
...,...,...,...,...
10662190,fcca183903b7,620586,0.002600,0.000845
10662191,fcca183903b7,620598,0.002680,0.000886
10662192,fcca183903b7,620610,0.002771,0.000965
10662193,fcca183903b7,620622,0.002990,0.001124


In [5]:
sub

,series_id,step,event,score,reduced_score
0,038441c925bb,333486,onset,8.141096,8.141096
1,038441c925bb,315894,onset,8.133041,8.133041
2,038441c925bb,281382,onset,8.129609,8.129609
3,038441c925bb,297558,onset,8.085974,8.085974
4,038441c925bb,39978,onset,8.005275,8.005275
...,...,...,...,...,...
259802,fe90110788d2,124590,onset,0.066447,0.066447
259803,fe90110788d2,317646,wakeup,0.066300,0.066300
259804,fe90110788d2,488418,onset,0.066289,0.066289
259805,fe90110788d2,522930,onset,0.066198,0.066198


In [5]:
labels = pd.read_csv(f"{CFG['dataset']['competition_dir']}/train_events.csv").dropna()
labels.head()

,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400


0         False
1         False
2         False
3         False
4         False
          ...  
389875    False
389876    False
389877    False
389878    False
389879    False
Name: timestamp, Length: 389880, dtype: bool

In [ ]:
sensor_df

In [13]:
for sid, oof_df in tqdm(oof.groupby("series_id")):
    sub_df = sub[sub["series_id"] == sid].sort_values("step").reset_index(drop=True)
    label_df = labels[labels["series_id"] == sid].sort_values("step").reset_index(drop=True)
    sensor_df = pd.read_csv(os.path.join(CFG["dataset"]["step_csv_dir"], f"{sid}.csv"))

    fig, axs = plt.subplots(3, 1, figsize=(20, 10))

    axs[0].plot(oof_df["step"], oof_df["wakeup_oof"], label="wakup")    
    axs[0].plot(oof_df["step"], oof_df["onset_oof"], label="onset")
    ax2 = axs[0].twinx()
    for night, night_df in label_df.groupby("night"):
        ax2.axvspan(night_df["step"].min(), night_df["step"].max(), color="red", alpha=0.1)    
    axs[0].scatter(sub_df["step"], sub_df["score"], color="red", s=7)
    switch_time = sensor_df[sensor_df["timestamp"].str[11:19] == "12:00:00"]["step"].values
    for switch in switch_time:
        axs[0].axvline(switch, color="black", alpha=0.5, ls="--")
    axs[0].grid()

    axs[1].plot(sensor_df["step"], sensor_df["anglez"],)
    axs[1].set_ylabel("anglez")
    ax2 = axs[1].twinx()
    for night, night_df in label_df.groupby("night"):
        ax2.axvspan(night_df["step"].min(), night_df["step"].max(), color="red", alpha=0.1)
    axs[1].grid()

    axs[2].plot(sensor_df["step"], sensor_df["enmo"],)
    axs[2].set_ylabel("enmo")
    ax2 = axs[2].twinx()
    for night, night_df in label_df.groupby("night"):
        ax2.axvspan(night_df["step"].min(), night_df["step"].max(), color="red", alpha=0.1)
    axs[2].grid()

    path = f"/kaggle/output/{CFG['execution']['exp_id']}/all_days/{sid}.png"
    os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.suptitle(f"sid: {sid}")
    plt.tight_layout()
    plt.savefig(path)
    plt.close()
    # plt.show()        
    # break

100%|██████████| 277/277 [05:59<00:00,  1.30s/it]
